In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
addres

In [ ]:


import pandas as pd
import numpy as np
import seaborn as sns
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
validation_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_validation_data.csv.xz")
#tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")

X_train= training_data.drop(['id','data_type','target'],axis=1)
y_train=training_data['target']

X_val = validation_data.drop(['id','data_type','target'],axis=1)
y_val = validation_data['target']

#X_test=tournament_data.drop(['id','data_type'],axis =  1 )


print("number of features {}".format(len(training_data.columns)-4))
#print("ratio of val to train {}".format(len(y_val)/len(y_train)))

number of features 310


In [ ]:
#from pandas.core.groupby.generic import DataFrameGroupBy
#from sklearn.preprocessing import MinMaxScaler
#def get_group_stats(df: pd.DataFrame) -> pd.DataFrame:
#        for group in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]:
#            cols = [col for col in df.columns if group in col]
#            df[f"feature_{group}_mean"] = df[cols].mean(axis=1)
#            df[f"feature_{group}_std"] = df[cols].std(axis=1)
#            df[f"feature_{group}_skew"] = df[cols].skew(axis=1)
#        return df

#def min_max_stats(df: pd.DataFrame) -> pd.DataFrame:
#        mini=MinMaxScaler()
#        for group in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]:
#            df[f"feature_{group}_mean"] = mini.fit_transform(df[f"feature_{group}_mean"].values.reshape(-1,1))
#            df[f"feature_{group}_std"] = mini.fit_transform(df[f"feature_{group}_std"].values.reshape(-1,1))
#            df[f"feature_{group}_skew"] = mini.fit_transform(df[f"feature_{group}_skew"].values.reshape(-1,1))
#        return df 

import scipy
from sklearn.preprocessing import LabelEncoder

def eraencode(df,col):
    lb=LabelEncoder()
    df[col]=lb.fit_transform(df[col])
    return df

def interpscale(arr,minv,maxv):
    return np.interp(arr,(np.min(minv),np.max(maxv)),(minv,maxv))


def _neutralize(df, preds, proportion):
    scores = preds
    exposures = df.values
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    return scores / scores.std()
def _normalize(df):
    X = (df.rank(method="first") - 0.5) / len(df)
    return scipy.stats.norm.ppf(X)
def normalize_and_neutralize(df, preds, feats, proportion=1.0):
    # Convert the scores to a normal distribution
    df[preds] = _normalize(df[preds])
    df[preds] = _neutralize(df, preds, feats, proportion)
    return df[preds]
def ar1(x):
    return np.corrcoef(x[:-1], x[1:])[0,1]

def autocorr_penalty(x):
    n = len(x)
    p = np.abs(ar1(x))
    return np.sqrt(1 + 2*np.sum([((n - i)/n)*p**i for i in range(1,n)]))

def smart_sharpe(x):
    return (np.mean(x)/(np.std(x, ddof=1) * autocorr_penalty(x)) * np.sqrt(12))

def numerai_sharpe(x):
    return ((np.mean(x) - 0.010415154) / np.std(x)) * np.sqrt(12)

def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]


# spread out target to 1, 2, 3, 4
train=eraencode(X_train,'era')
val=eraencode(X_val,'era')
#test=eraencode(X_test,'era')


y_train_int= (y_train *4).astype(np.int32)
y_val_int = (y_val*4).astype(np.int32)



#trainm= min_max_stats(train)
#valm=min_max_stats(val)

In [ ]:
sd = train.std().sort_values(ascending = False)
sd

era                      34.907755
feature_wisdom29          0.353596
feature_charisma83        0.353596
feature_intelligence4     0.353596
feature_intelligence1     0.353596
                           ...    
feature_strength17        0.284223
feature_charisma47        0.265371
feature_charisma25        0.265371
feature_strength38        0.180383
feature_strength20        0.180383
Length: 311, dtype: float64

In [ ]:
highsdcol=sd[sd > 0.345].index

In [ ]:
train2=train[highsdcol]
val2=val[highsdcol]
#test2=test[highsdcol]

In [ ]:
!pip install catboost lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.5 MB/s eta 0:00:00


In [ ]:
##Xu=[1,2,3,4,5,6,7,8,9,10]
#yu=[0,1,0,1,0,1,0,1,0,1]
#param_distribution = {
  #  'l2_leaf_reg': [1, 3, 5, 7, 9]
#}
#est=CatBoostRegressor()
#e=est.grid_search(param_distribution,Xu,yu)


In [ ]:
#est.predict([13])

In [ ]:
#e['params']

In [ ]:
#est.get_all_params()

In [ ]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor as lgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
%timeit
def ensemble_train_val_predict(init_paramsl,init_paramsc,gridl,gridc,X_train,y_train,X_val,y_val,Npochs = 1,N_Fold= 10,cvl=4,models_app=True,plt=False,neut=False):
    
    cbr=CatBoostRegressor(**init_paramsc)
    lg = lgb(**init_paramsl)
    
    if models_app:
        models=dict()
    
    pred_c=np.zeros(len(X_val['era'].values.reshape(-1,1)))
    pred_l=np.zeros(len(X_val['era'].values.reshape( -1,1)))
  #  pred_lt=np.zeros( len(X_train['era'].values.reshape( -1,1)))
    
    #k = KFold(N_Fold)
    tss=TimeSeriesSplit(n_splits=N_Fold)

    
    for i in range(Npochs):
        for fold_, (tr_idx,val_idx) in enumerate(tss.split(X_train,y_train)):
            
            glg= GridSearchCV(estimator=lg,param_grid=gridl)
            glg.fit(X_train.iloc[tr_idx],y_train.iloc[tr_idx])
            
            cbr.grid_search(gridc, X_train,y_train,plot=plt) # grid search updates the parameters of the class instance no worries to update with pointer from dict
            cbr.set_params()
            
       #     
            print("for LightGBMReg @ {} Fold the Training Corr is {}".format(fold_,correlation_score(y_train.iloc[tr_idx],glg.predict(X_train.iloc[tr_idx]))))
            print("for LightGBMReg @ sub {} Fold the Val Corr is {}".format(fold_,correlation_score(y_train.iloc[val_idx],glg.predict(X_train.iloc[val_idx]))))


            print("for CatBoostReg @ {} Fold the Training Corr is {}".format(fold_,correlation_score(y_train.iloc[tr_idx],cbr.predict(X_train.iloc[tr_idx]))))
            print("for CatBoostReg @ sub {} Fold the Val Corr is {}".format(fold_,correlation_score(y_train.iloc[val_idx],cbr.predict(X_train.iloc[val_idx]))))
      #  print("for light gbm the Val Corr is {}".format(correlation_score(y_train,glg.predict(X_train))))            
            pred_l+= glg.predict(X_val)/N_Fold 
            pred_c+= cbr.predict(X_val)/N_Fold
            #pred_l+= glg.predict(X_test)/N_Fold
            
         #   pred_lt += glg.predict(X_train)
            if models_app:
                cur_key1='LightGBMReg Model @ TS Fold'+str(fold_)
                cur_key2='CatBoostReg Model @ TS Fold' + str(fold_)
                # append to dict for reproduction X_test is too much RAM seperate process VM
                models[cur_key1]=(glg)
                models[cur_key2]=(cbr)
    # end of epoch final func protocol 
    
    predCL=np.mean([pred_c,pred_l],axis = 0)
    
 #   pred2t=interpscale(pred_lt,0 ,1 )
 #   print(" for light gbm the Training Corr is { }".format(correlation_score(y_train,pred2t)))
    print(" for LightGBMReg the Val Corr is {}".format(correlation_score(y_val,pred_l)))
    print(" for CatBoostReg the Val Cross is {}".format(correlation_score(y_val,pred_c)))
    print(" for bagged ensemble of LightGBMReg + CatBoostReg the Val Cross is {}".format(correlation_score(y_val,predCL)))


    if neut:
        prop=np.arange(0.05,1,0.05)
        
        predneut_l = [correlation_score(y_val,_neutralize(X_val,pred_l,l)) l for l in prop ]
        neutindex_l=np.where(predneut_l==np.max(predneut_l ))
        print(" for LightGBMReg Neutralized the Val Corr is {} with proportion {}".format(predneut_l[neutindex_l], prop[neutindex_l]))


        predneut_c = [correlation_score(y_val,_neutralize(X_val,pred_c,c)) c for c in prop ]
        neutindex_c=np.where(predneut_c==np.max(predneut_c ))
        print(" for CatBoostReg Neutralized the Val Corr is {} with proportion {}".format(predneut_c[neutindex_c], prop[neutindex_c]))


        predneut_cl = [correlation_score(y_val,_neutralize(X_val,pred_CL,cl)) cl for cl in prop ]
        neutindex_cl=np.where(predneut_cl==np.max(predneut_cl ))
        print(" for CatBoostReg Neutralized the Val Corr is {} with proportion {}".format(predneut_cl[neutindex_cl], prop[neutindex_cl]))
        pass

       

    # device which model to save afterwards
    if models_app:
        return models

#def submit_predictions(modeldict,model_key,X_test):
#    clf= modeldict[model_key]





init_paramsl = {
    'metric':'mse'
    
}

init_paramsc={ 'loss_function':'RMSE'}

gridc = {
    'learning_rate': [0.03, 0.01, 0.03, 0.009],
  #  'bagging_temperature': [0, 1,5,10],
     'max_depth': np.arange(5,20,5),
     'n_estimators':np.arange(100,250,50),
  #   'num_leaves':np.arange(25,40,5)
     }

gridl ={'reg_alpha': [0.1, 0.5],
    'lambda_l1': [0, 1, 1.5],
    'lambda_l2': [0, 1],
    'max_depth': np.arange(5,20,5),
    'n_estimators':np.arange(100,250,50),
    'learning_rate': [0.03, 0.01, 0.03, 0.009],
    'num_leaves':np.arange(25,40,5)
    }


resdict=ensemble_train_val_predict(init_paramsl,init_paramsc,gridl,gridc,train2,y_train,val2,y_val)